# Lab | Web Scraping Single Page (GNOD part 1)

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = "http://www.popvortex.com/music/charts/top-100-songs.php"

In [3]:
response = requests.get(url)
response.status_code

200

In [4]:
soup = BeautifulSoup(response.content, "html.parser")

In [5]:
#soup

In [6]:
#print(soup.prettify())

In [7]:
soup.p

<p>Chart of <strong>iTunes top 100 songs</strong> downloaded at the Apple iTunes music store today. The iTunes top 100 songs chart list countdown includes the hottest and most popular music hits and singles from all musical genres including today's hot <a href="top-pop-songs.php">pop</a>, <a href="top-rap-songs.php">rap</a>, <a href="top-rock-songs.php">rock</a>, and <a href="top-country-songs.php">country</a> songs.</p>

In [8]:
soup.select('p')
for p in soup.select('p'):
    p.get_text()

In [9]:
song_info = []

for song in soup.find_all('p', class_='title-artist'):
    song_name = song.find('cite', class_='title').get_text()
    artist_name = song.find('em', class_='artist').get_text()
    song_info.append((song_name, artist_name))

#print("Songs:", song_info)

In [10]:
songs = []
artists = []

for li in soup.select("body > div.container > div:nth-child(4) > div.col-xs-12.col-md-8 > div.chart-wrapper > div "):
    for line in li.select('cite.title'):
        songs.append(line.get_text())
    for line in li.select('em.artist'):
        artists.append(line.get_text())

In [11]:
top_chart_songs = pd.DataFrame({"song": songs, "artist": artists})
top_chart_songs

,song,artist
0,Lovin On Me,Jack Harlow
1,Lil Boo Thang,Paul Russell
2,White Horse,Chris Stapleton
3,I Remember Everything (feat. Kacey Musgraves),Zach Bryan
4,Need A Favor,Jelly Roll
...,...,...
95,Autumn Comes Dry,Luminara Stellaris
96,Last Christmas (Single Version),Wham!
97,In My Life,The Beatles
98,TRUCK BED,HARDY


In [12]:
# Get User Input
user_input_song = input("Enter a song: ")

# Convert the user input and song names to lowercase for case-insensitive comparison
user_input_song_lower = user_input_song.lower()
top_chart_songs_lower = top_chart_songs.apply(lambda x: x.str.lower())

# Check if the user-inputted song is in the top chart songs list (case-insensitive)
if user_input_song_lower in top_chart_songs_lower['song'].values:
    # Filter out the user-inputted song from the DataFrame
    top_chart_songs_without_input = top_chart_songs[
        top_chart_songs_lower['song'] != user_input_song_lower
    ]

    # Recommend a random song from the filtered DataFrame
    recommended_song = top_chart_songs_without_input.sample().squeeze()

    print(f"You might also like: {recommended_song['song']} by {recommended_song['artist']}")
else:
    print("We can't provide a recommendation right now.")

Enter a song: exes
You might also like: Around Me the Cold Night by Luminara Stellaris


# Lab | Web Scraping Multiple Pages

In [13]:
url = "https://en.wikipedia.org/wiki/List_of_most-streamed_songs_in_the_United_Kingdom"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [14]:
table = soup.find('table', class_='wikitable')
#print(table)

In [15]:
songs = []
uk_artists = []
row_count = 0
for row in table.find_all('tr')[1:]:
    columns = row.find_all(['td','th'])
    songs.append(columns[1].text.strip())
    uk_artists.append(columns[2].text.strip())
    
    row_count+=1
    if row_count >= 48 :
        break

uk_songs = pd.DataFrame({'Title':songs, 'Artists':uk_artists})
uk_songs

,Title,Artists
0,"""Someone You Loved""",Lewis Capaldi
1,"""Shape of You""",Ed Sheeran
2,"""Perfect"" / ""Perfect Duet"" / ""Perfect Symphony""",Ed Sheeran / Ed Sheeran featuring Beyoncé / Ed...
3,"""Shotgun""",George Ezra
4,"""Dance Monkey""",Tones and I
5,"""Blinding Lights""",The Weeknd
6,"""Mr. Brightside""",The Killers
7,"""Baby Shark""",Pinkfong
8,"""Despacito"" /""Despacito (Remix)""",Luis Fonsi featuring Daddy Yankee / Luis Fonsi...
9,"""Shallow""",Lady Gaga and Bradley Cooper
